### Zigbang 원룸 매물 데이터 수집

In [50]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [ ]:
url = 'https://apis.zigbang.com/v2/search?leaseYn=N&q=망원동&serviceType=원룸'
response = requests.get(url)
response

In [ ]:
lat = response.json()['items'][0]['lat']
lng = response.json()['items'][0]['lng']
lat, lng

#### 2. 위도 경도로 geohash 알아내기

In [51]:
import geohash2

In [ ]:
# precision : 영역크기, 커질수록 영역이 작아진다.
geohash = geohash2.encode(lat, lng, precision=5)
geohash

#### 3. geohash로 매물 아이디 가져오기

In [ ]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

In [ ]:
items = response.json()['items']
print('전처리 이전')
display(items[:5])
print('\n전처리 이후')
ids = []
for item in items:
    ids.append(item['item_id'])
len(ids), ids[:5]

#### 4. 매물 아이디로 매물 정보 가져오기

In [ ]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {'domain': 'zigbang', 'item_ids': ids}
response = requests.post(url, params)
response

In [ ]:
response.json()['items'][0]

In [ ]:
# tip : 직방은 요청 id가 1000개를 초과하면 에러가 발생한다.
items = response.json()['items']
df = pd.DataFrame(items)
df.head()

In [ ]:
df.columns

In [ ]:
columns = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 
           'floor', 'floor_string', 'building_floor', 'title', 'service_type', 
           'manage_cost', 'address1']
df = df[columns]

In [ ]:
df.head()

In [ ]:
df = df.loc[df['address1'].str.contains('망원동')]
df

In [ ]:
df = df.reset_index(drop=True)
df

#### 5. 함수화

In [53]:
def oneroom(addr):
    # 1. 망원동의 위도, 경도 구하기
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    lat = response.json()['items'][0]['lat']
    lng = response.json()['items'][0]['lng']
    
    # 2. geohash 값 구하기
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash 위치의 매물아이디 찾아오기
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    #4. response 값에서 아이디값만 가져오기
    items = response.json()['items']
    ids = []
    for item in items:
        ids.append(item['item_id'])
        
    #5. 해당 아이디의 매물 정보 가져와서 필요한 데이터 뽑아내기
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain': 'zigbang', 'item_ids': ids}
    response = requests.post(url, params)
    
    items = response.json()['items']
    df = pd.DataFrame(items)
    
    columns = ['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']
    df = df[columns]
    df = df.loc[df['address1'].str.contains(addr)]
    return df.reset_index(drop=True)

In [46]:
%%time
result = oneroom('마포구 합정동')
result.head()

Wall time: 503 ms


,item_id,title,deposit,rent,address1,size_m2,manage_cost
0,37582011,합정역3분 이내 보증보험 가능한 신축 투룸,33000,0,서울시 마포구 합정동,35.53,0
1,37582669,"합정역 바로입주, 보증보험 가능한 투룸 전세",33000,0,서울시 마포구 합정동,35.53,8
2,37587014,HUG신혼부부전세ㅣ안심전세ㅣ보증보험ㅣ합정역 4분ㅣ신축풀옵션,33000,0,서울시 마포구 합정동,35.53,0
3,37615982,합정역 도보5분 버팀목대출 가능한 2룸신축빌라,30000,0,서울시 마포구 합정동,35.53,8
4,37699687,v안심전세대출 v보증보험v 합정 코앞 신축 투룸v,33000,0,서울시 마포구 합정동,35.53,0


In [42]:
%whos

Variable   Type         Data/Info
---------------------------------
columns    list         n=12
df         DataFrame         item_id sales_type  <...>         1  서울시 마포구 망원동  
geohash    str          wydjx
geohash2   module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
ids        list         n=581
item       dict         n=3
items      list         n=581
lat        float        37.556785583496094
lng        float        126.9013442993164
oneroom    function     <function oneroom at 0x0000019ADAD21288>
params     dict         n=2
pd         module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module       <module 'requests' from '<...>\\requests\\__init__.py'>
response   Response     <Response [200]>
url        str          https://apis.zigbang.com/v2/items/list


In [48]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [49]:
%whos

Interactive namespace is empty.


In [52]:
%whos

Variable   Type      Data/Info
------------------------------
geohash2   module    <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
pd         module    <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module    <module 'requests' from '<...>\\requests\\__init__.py'>


In [54]:
%whos

Variable   Type        Data/Info
--------------------------------
geohash2   module      <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
oneroom    function    <function oneroom at 0x0000019ADA94A948>
pd         module      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module      <module 'requests' from '<...>\\requests\\__init__.py'>


In [55]:
%%time
result = oneroom('마포구 합정동')
result.head()

Wall time: 848 ms


,item_id,title,deposit,rent,address1,size_m2,manage_cost
0,37582011,합정역3분 이내 보증보험 가능한 신축 투룸,33000,0,서울시 마포구 합정동,35.53,0
1,37582669,"합정역 바로입주, 보증보험 가능한 투룸 전세",33000,0,서울시 마포구 합정동,35.53,8
2,37587014,HUG신혼부부전세ㅣ안심전세ㅣ보증보험ㅣ합정역 4분ㅣ신축풀옵션,33000,0,서울시 마포구 합정동,35.53,0
3,37615982,합정역 도보5분 버팀목대출 가능한 2룸신축빌라,30000,0,서울시 마포구 합정동,35.53,8
4,37699687,v안심전세대출 v보증보험v 합정 코앞 신축 투룸v,33000,0,서울시 마포구 합정동,35.53,0


In [56]:
%whos

Variable   Type         Data/Info
---------------------------------
geohash2   module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
oneroom    function     <function oneroom at 0x0000019ADA94A948>
pd         module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module       <module 'requests' from '<...>\\requests\\__init__.py'>
result     DataFrame         item_id             <...>\n\n[65 rows x 7 columns]


#### 6. 모듈화

In [57]:
%%writefile zigbang.py
import pandas as pd
import requests
import geohash2

def oneroom(addr):
    # 1. 망원동의 위도, 경도 구하기
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    lat = response.json()['items'][0]['lat']
    lng = response.json()['items'][0]['lng']
    
    # 2. geohash 값 구하기
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # 3. geohash 위치의 매물아이디 찾아오기
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    #4. response 값에서 아이디값만 가져오기
    items = response.json()['items']
    ids = []
    for item in items:
        ids.append(item['item_id'])
        
    #5. 해당 아이디의 매물 정보 가져와서 필요한 데이터 뽑아내기
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain': 'zigbang', 'item_ids': ids}
    response = requests.post(url, params)
    
    items = response.json()['items']
    df = pd.DataFrame(items)
    
    columns = ['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']
    df = df[columns]
    df = df.loc[df['address1'].str.contains(addr)]
    return df.reset_index(drop=True)

Writing zigbang.py
